# Matrix Assembly
After generating the mesh and defining the domain properties (source function, diffusion coefficient, etc.), we must assemble the linear system $A u = f$, where $A$ is an $N \times N$ matrix, $f$ is an $N$-element vector representing the source, and $u$ is the unknown.

In a different notebook, we showed that each element of the mesh contributes to the $A$ matrix and $f$ vector by an elementary contribution $A_{e_i}$ and $f_{e_i}$. Our task is now to assemble these elementry contributions into the global matrices in an efficient manner. In this notebook, we will show different methods of performing this assembly.

In [1]:
using LinearAlgebra, SparseArrays
using StructArrays

using BenchmarkTools

Using `generate_mesh` we can generate an 1D mesh on the domain $[0, 1]$ with an arbitrary number of elements $N_{el}$. This will be used for testing and benchmarking the matrix assembly functions.

In [2]:
struct Element
    p1::Float64
    p2::Float64
    e1::Int64
    e2::Int64
end 

function generate_mesh(Nel)
    x = 0:1/Nel:1;

    # Collect points (left and right nodes for each element) and edges (connectivity: indices of left and right nodes)
    N = length(x);
    points = collect( [x[i], x[i+1]] for i in 1:N-1) 
    edges  = collect( [i, i+1] for i in 1:N-1); 

    elem_ids = 1:length(edges)
    e_group = ones(size(edges));
    
    #..Set the source function 
    fsource(group_id) = 1;
    fsource_elem = map(fsource, e_group);

    # Generate a struct array with the required data per element
    mesh = StructArray{Element}((x[1:end-1], x[2:end], Vector(1:N-1), Vector(2:N)))

    return mesh, N, elem_ids, fsource_elem
end

generate_mesh (generic function with 1 method)

## Triple Loop
The most naive implementation of matrix assembly is to loop over all the elements and for each element calculate the the local contribution and add it to the global matrix. This addition can be done in two ways:
- By looping over the rows and columns of the local contributions (need two extra loops, hence triple loop assembly)
- By adding the local contribution directly using proper matrix indexing (single loop, see below)

Note that this approach has poor performance because a complete (dense) $N \times N$ matrix must be allocated. This is quite a waste of memory because $A$ is sparse. Defining $A$ as sparse, e.g. using `spzeros` will definitely not improve performance because it is very expensive to perform operations on only parts of a sparse matrix, especially because it will change the sparsity pattern in this case.

In [3]:
function assemble_matrices(mesh, N, fsource_elem)
    #..Initialize global matrix and right-hand side value 
    A = zeros(Float64, N, N);
    f = zeros(Float64, N); 

    #..Perform loop over elements and assemble global matrix and vector 
    @inbounds for (i, el) = enumerate(mesh)
        xl = el.p1; xr = el.p2;
        j  = el.e1; k  = el.e2;        
        h = xr - xl;

        # Compute the elementary contributions (P1 elements)
        floc = fsource_elem[i] * h / 2 * [1; 1];
        Aloc = 1 / h * [1 -1; -1 1];

        #....perform loop over nodes of the current element
        #....and add local contribution Aloc to global matrix entity A
        nodes = [j, k]
        for x = 1:2
            I     = nodes[x];
            f[I] += floc[x]
            for y = 1:2
                J = nodes[y];
                A[I,J] += Aloc[x, y]
            end
        end

    end

    #..handle the boundary conditions in the matrix and right-hand side vector 
    A[1,:]   = zeros(N);  A[1,1]     = 1;      f[1]   = 0;
    A[end,:] = zeros(N);  A[end,end] = 1;      f[end] = 0;
    
    return A, f
end

assemble_matrices (generic function with 1 method)

In [4]:
mesh, N, elem_ids, fsource_elem = generate_mesh(10000);

@benchmark A, f = assemble_matrices(mesh, N, fsource_elem)

BenchmarkTools.Trial: 24 samples with 1 evaluation.
 Range (min … max):  132.372 ms … 304.859 ms  ┊ GC (min … max):  0.00% … 35.00%
 Time  (median):     197.116 ms               ┊ GC (median):    16.21%
 Time  (mean ± σ):   211.343 ms ±  59.385 ms  ┊ GC (mean ± σ):  25.09% ± 20.86%

  ▁             █                      ▁                      ▄  
  █▆▆▁▁▁▆▁▆▁▁▁▁▁█▁▁▁▁▆▁▁▆▆▁▁▁▁▁▁▁▁▁▁▁▁▆█▆▆▁▁▁▁▁▁▁▁▁▁▁▆▆▆▁▁▁▁▁▁█ ▁
  132 ms           Histogram: frequency by time          305 ms <

 Memory estimate: 767.90 MiB, allocs estimate: 60009.

In [5]:
@btime A, f = assemble_matrices(mesh, N, fsource_elem)

  135.945 ms (60009 allocations: 767.90 MiB)


([1.0 0.0 … 0.0 0.0; -10000.0 20000.0 … 0.0 0.0; … ; 0.0 0.0 … 20000.0000000022 -10000.0000000011; 0.0 0.0 … 0.0 1.0], [0.0, 0.0001, 9.999999999999999e-5, 0.0001, 0.00010000000000000002, 9.999999999999996e-5, 9.999999999999999e-5, 0.00010000000000000005, 9.999999999999999e-5, 9.999999999999999e-5  …  9.999999999998899e-5, 9.999999999998899e-5, 9.999999999998899e-5, 0.0001000000000000445, 0.0001000000000000445, 9.999999999998899e-5, 9.999999999998899e-5, 9.999999999998899e-5, 9.999999999998899e-5, 0.0])

## Single Loop
The single loop assembly is very similar to the triple loop shown above. Instead of looping over the local contribution, we add it into the matrices directly:
```julia
f[nodes]        += floc;
A[nodes, nodes] += Aloc;
```
where `nodes` is a two-element vector containing the left and right node of the element: $[x_l, x_r]$. Although this is more expressive than the triple loop, the benchmark shows that it decreases performance by a little bit.

In [6]:
function assemble_matrices(mesh, N, fsource_elem)    
    #..Initialize global matrix and right-hand side value 
    A = zeros(Float64, N, N);
    f = zeros(Float64, N); 

    #..Perform loop over elements and assemble global matrix and vector 
    @inbounds for (i, el) = enumerate(mesh)
        xl = el.p1; xr = el.p2;
        j  = el.e1; k  = el.e2;        
        h = xr - xl;

        # Compute the elementary contributions (P1 elements)
        floc = fsource_elem[i] * h / 2 * [1; 1];
        Aloc = 1 / h * [1 -1; -1 1];

        # Add local contributions to the global matrix
        nodes = [j, k];
        f[nodes]        += floc;
        A[nodes, nodes] += Aloc;
    end

    #..handle the boundary conditions in the matrix and right-hand side vector 
    A[1,:]   = zeros(N);  A[1,1]     = 1;      f[1]   = 0;
    A[end,:] = zeros(N);  A[end,end] = 1;      f[end] = 0;
    
    return A, f
end

assemble_matrices (generic function with 1 method)

In [7]:
mesh, N, elem_ids, fsource_elem = generate_mesh(10000);

@benchmark A, f = assemble_matrices(mesh, N, fsource_elem)

BenchmarkTools.Trial: 21 samples with 1 evaluation.
 Range (min … max):  136.073 ms … 340.459 ms  ┊ GC (min … max):  0.00% … 51.80%
 Time  (median):     188.546 ms               ┊ GC (median):     1.93%
 Time  (mean ± σ):   217.885 ms ±  68.220 ms  ┊ GC (mean ± σ):  26.10% ± 21.99%

      ▃      █   ▃                                               
  ▇▇▁▇█▁▁▇▁▁▁█▇▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▇▁▇▁▁▇▁▁▁▁▁▇▁▁▁▇▁▇▁▇▁▁▁▇▁▁▁▁▇ ▁
  136 ms           Histogram: frequency by time          340 ms <

 Memory estimate: 771.26 MiB, allocs estimate: 100009.

In [8]:
@btime A, f = assemble_matrices(mesh, N, fsource_elem)

  134.222 ms (100009 allocations: 771.26 MiB)


([1.0 0.0 … 0.0 0.0; -10000.0 20000.0 … 0.0 0.0; … ; 0.0 0.0 … 20000.0000000022 -10000.0000000011; 0.0 0.0 … 0.0 1.0], [0.0, 0.0001, 9.999999999999999e-5, 0.0001, 0.00010000000000000002, 9.999999999999996e-5, 9.999999999999999e-5, 0.00010000000000000005, 9.999999999999999e-5, 9.999999999999999e-5  …  9.999999999998899e-5, 9.999999999998899e-5, 9.999999999998899e-5, 0.0001000000000000445, 0.0001000000000000445, 9.999999999998899e-5, 9.999999999998899e-5, 9.999999999998899e-5, 9.999999999998899e-5, 0.0])

## Single Loop with Pre-allocation
If the matrices need to be assembled many times (for example, for each linear solver step in a non-linear problem), it is very inefficient to allocate a new matrix for each step. Instead, we can _preallocate_ $A$ and $f$. To save memory, we let $A$ be a sparse matrix in this case. In the preallocation, we also create the correct sparsity pattern based on the mesh.

In [9]:
function prealloc_matrix(mesh, N)
    # Initialize vectors for sparse matrix construction
    I = zeros(Int64, 4 * (N - 1))
    J = similar(I);
    V = zeros(Float64, length(I));
    
    # Allocate f vector
    f = zeros(N)
    
    @inbounds for (i, el) = enumerate(mesh)
        idx = 4*(i-1) + 1 : 4*i;
        j  = el.e1; k  = el.e2;
        
        I[idx] = [j, k, j, k];
        J[idx] = [j, j, k, k];
        V[idx] = [1 1 1 1];      # The values are unimportant, just for allocating sparsity pattern
    end
    
    # Create sparse matrix and zero all the entries
    # The sparsity pattern stays intact
    A = sparse(I, J, V)
    fill!(A.nzval, 0)
    
    return A, f;
end

prealloc_matrix (generic function with 1 method)

Next, we can use an in-place function to insert the correct values in $A$ and $f$. A triple loop is used here because benchmarking has shown that this is much more efficient (for sparse matrices especially) than using the vector indexing in the single-loop assembly.

Note that we must fill the matrix and vector with zero before we start assembly.

In [10]:
function assemble_matrices!(A, f, mesh, fsource_elem)
    fill!(A.nzval, 0)
    fill!(f, 0)
    
    Aloc = zeros(2, 2)
    floc = zeros(2)
    
    #..Perform loop over elements and assemble global matrix and vector 
    @inbounds for (i, el) = enumerate(mesh)
        xl = el.p1; xr = el.p2;
        j  = el.e1; k  = el.e2;        
        h = xr - xl;

        # Compute the elementary contributions (P1 elements)
        floc[:] = fsource_elem[i] * h / 2 * [1; 1];
        Aloc[:] = 1 / h * [1 -1; -1 1];

        # Add local contributions to the global matrix
        nodes = [j, k];
        for x = 1:2
            I     = nodes[x];
            f[I] += floc[x]
            for y = 1:2
                J = nodes[y];
                A[I,J] += Aloc[x, y]
            end
        end
    end
    
    #..handle the boundary conditions in the matrix and right-hand side vector 
    A[1,:]   = zeros(N);  A[1,1]     = 1;      f[1]   = 0;
    A[end,:] = zeros(N);  A[end,end] = 1;      f[end] = 0;
end

assemble_matrices! (generic function with 1 method)

In [11]:
mesh, N, elem_ids, fsource_elem = generate_mesh(10000);
A, f = prealloc_matrix(mesh, N);

@benchmark assemble_matrices!(A, f, mesh, fsource_elem)

BenchmarkTools.Trial: 807 samples with 1 evaluation.
 Range (min … max):  3.637 ms … 115.854 ms  ┊ GC (min … max):  0.00% … 96.20%
 Time  (median):     5.639 ms               ┊ GC (median):     0.00%
 Time  (mean ± σ):   6.193 ms ±   8.579 ms  ┊ GC (mean ± σ):  11.99% ±  8.19%

   ▁▄              ▃▅▆█▆▄▃                                     
  ▅███▇▅▅▄▆▅▄▅▅█▇▄▅█████████▆▄▁▆▁▁▁▄▄▁▁▁▁▁▁▄▁▁▁▁▁▁▁▁▁▁▁▄▁▁▁▁▅ █
  3.64 ms      Histogram: log(frequency) by time      9.47 ms <

 Memory estimate: 5.96 MiB, allocs estimate: 60037.

In [12]:
@btime assemble_matrices!(A, f, mesh, fsource_elem)

  3.686 ms (60037 allocations: 5.96 MiB)


0

Observe that this is much more efficient than the other two approaches shown until now. This is due to
1. Preallocation of the matrix
2. Usage of sparse matrices

## Single Loop with Sparse
We can also use the `sparse(I, J, V)` function to build sparse matrices. The main advantage of this is that it does not require us to perform any expensive operations on the sparse matrix (remember: indexing a sparse matrix is much less efficient than indexing a dense matrix).

Instead, we build three arrays containing the indices of the non-zero entries (`I` and `J`) and their values (`V`). This can then be converted into a sparse matrix using the standard function `sparse(I, J, V)`. This approach is slightly less efficient than using preallocation.

In [13]:
function assemble_matrices(mesh, N, fsource_elem)
    # Initialize vectors for sparse matrix construction
    I = zeros(Int64, 4 * (N - 1))
    J = similar(I);
    V = zeros(Float64, length(I));
    
    # Initialize right-hand side vector
    f = zeros(Float64, N); 
    
    #..Perform loop over elements and assemble global matrix and vector 
    @inbounds for (i, el) = enumerate(mesh)
        idx = 4*(i-1) + 1 : 4*i;
        xl = el.p1; xr = el.p2;
        j  = el.e1; k  = el.e2;
        
        h  = xr - xl;
        
        #
        f[[j, k]] += fsource_elem[i] * h / 2 * [1; 1];
        
        # Matrix contribution
        I[idx] = [j, k, j, k];
        J[idx] = [j, j, k, k];
        V[idx] = 1/h * [1 -1 -1 1];
    end
    
    A = sparse(I, J, V)
    
    #..handle the boundary conditions in the matrix and right-hand side vector 
    A[1,:]   = zeros(N);  A[1,1]     = 1;      f[1]   = 0;
    A[end,:] = zeros(N);  A[end,end] = 1;      f[end] = 0;
    
    return A, f
end

assemble_matrices (generic function with 1 method)

In [14]:
mesh, N, elem_ids, fsource_elem = generate_mesh(10000);

@benchmark A, f = assemble_matrices(mesh, N, fsource_elem)

BenchmarkTools.Trial: 609 samples with 1 evaluation.
 Range (min … max):  3.706 ms … 100.576 ms  ┊ GC (min … max):  0.00% … 92.70%
 Time  (median):     7.780 ms               ┊ GC (median):     0.00%
 Time  (mean ± σ):   8.207 ms ±   9.771 ms  ┊ GC (mean ± σ):  13.77% ± 10.67%

    █                                                          
  ▅▃█▄▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂ ▂
  3.71 ms         Histogram: frequency by time        88.3 ms <

 Memory estimate: 11.14 MiB, allocs estimate: 90061.

In [15]:
@btime A, f = assemble_matrices(mesh, N, fsource_elem)

  3.641 ms (90061 allocations: 11.14 MiB)


(sparse([1, 2, 1, 2, 3, 2, 3, 4, 3, 4  …  9998, 9999, 9998, 9999, 10000, 9999, 10000, 10001, 10000, 10001], [1, 1, 2, 2, 2, 3, 3, 3, 4, 4  …  9998, 9998, 9999, 9999, 9999, 10000, 10000, 10000, 10001, 10001], [1.0, -10000.0, 0.0, 20000.0, -10000.0, -10000.0, 20000.000000000004, -10000.000000000004, -10000.000000000004, 20000.0  …  20000.0000000022, -10000.0000000011, -10000.0000000011, 20000.0000000022, -10000.0000000011, -10000.0000000011, 20000.0000000022, 0.0, -10000.0000000011, 1.0], 10001, 10001), [0.0, 0.0001, 9.999999999999999e-5, 0.0001, 0.00010000000000000002, 9.999999999999996e-5, 9.999999999999999e-5, 0.00010000000000000005, 9.999999999999999e-5, 9.999999999999999e-5  …  9.999999999998899e-5, 9.999999999998899e-5, 9.999999999998899e-5, 0.0001000000000000445, 0.0001000000000000445, 9.999999999998899e-5, 9.999999999998899e-5, 9.999999999998899e-5, 9.999999999998899e-5, 0.0])

## No Loop
We can make use of the functional programming methods `map` and `reduce` to assemble the $A$ matrix without needing any loops.

The performance of this code is discussed on Discourse: https://discourse.julialang.org/t/different-performance-between-reduce-map-and-mapreduce/85149

Note: still missing assembly of $f$ vector

In [16]:
function generate_mesh(Nel)
    x = 0:1/Nel:1;

    # Collect points (left and right nodes for each element) and edges (connectivity: indices of left and right nodes)
    N = length(x);
    points = collect( [x[i], x[i+1]] for i in 1:N-1) 
    edges  = collect( [i, i+1] for i in 1:N-1); 

    elem_ids = 1:length(edges)
    e_group = ones(size(edges));
    
    #..Set the source function 
    fsource(group_id) = 1;
    fsource_elem = map(fsource, e_group);
    
    return x, edges, N, elem_ids, fsource_elem
end

generate_mesh (generic function with 1 method)

In [17]:
# Compute the length h of an element, given its ID and the global list of elements and points
function compute_element_area(elem_id, e, p)
    area_id = p[e[elem_id][2]] - p[e[elem_id][1]]
    return area_id
end

compute_element_area (generic function with 1 method)

In [22]:
function assemble_matrices(elem_area, elems, x, N)        
    # Generate index vectors
    I = reduce(vcat, view.(elems, Ref([1, 2, 1, 2])))
    J = reduce(vcat, view.(elems, Ref([1, 1, 2, 2])))
    
    # Generate matrix contributions
    Atempl = [1, -1, -1, 1];
    V = reduce(vcat, map(h -> Atempl / h, elem_area));
    
    return sparse(I, J, V, N, N)
end

assemble_matrices (generic function with 2 methods)

In [23]:
x, edges, N, elem_ids, fsource_elem = generate_mesh(10000);

# Calculate element length h
elem_area = map(elem_id -> compute_element_area(elem_id, edges, x), elem_ids);

@benchmark A = assemble_matrices(elem_area, edges, x, N)

BenchmarkTools.Trial: 2136 samples with 1 evaluation.
 Range (min … max):  1.135 ms … 76.296 ms  ┊ GC (min … max):  0.00% … 96.80%
 Time  (median):     2.401 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   2.335 ms ±  3.757 ms  ┊ GC (mean ± σ):  11.35% ±  6.91%

   ▇                              ▁▂█▅                        
  ███▅▅▄▃▃▂▃▅▆▄▃▂▃▂▂▂▂▂▂▂▂▂▄▄▃▂▂▂▆█████▅▅▄▄▆▅▃▃▂▃▂▂▂▂▁▂▁▂▂▂▂ ▃
  1.13 ms        Histogram: frequency by time        3.36 ms <

 Memory estimate: 4.05 MiB, allocs estimate: 10041.

In [24]:
@btime A = assemble_matrices(elem_ids, edges, x, length(x))

  1.142 ms (10042 allocations: 4.05 MiB)


10001×10001 SparseMatrixCSC{Float64, Int64} with 30001 stored entries:
⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦